## Finding configuration and other necessary data
- Configuration and definitions files that are distributed with the core tools are found in the **`etc`** directory
- Other data files such as audio templates should be part of the external experimental setup (e.g. in the **`evoclearn-raw`** sub-project). For these examples we have included some data files in **`examples/data`**.

In [1]:
from os import path
etcdir = path.join("..", "etc")
datadir = "data"

## Initialise VocalTractLab with a speaker
Needed if we want to synthesise audio

In [2]:
from evoclearn.core import vocaltractlab as vtl
vtl.initialise(path.join(etcdir, "JD2_speaker.xml"))

<CDLL '/home/demitasse/workspace/evoclearn/evl_core/evoclearn/core/VocalTractLabApi.so', handle 2f84400 at 0x7fdf647dca30>

## Import some Python and evoclearn-core components

In [3]:
import functools
import itertools

import librosa

from evoclearn.core.samplers import UniformProbLink
from evoclearn.core.io import load_bounds, read_durations_from_textgrid, read_intervals_from_textgrid, wav_write
from evoclearn.core.utils import KeepLastValue
from evoclearn.core import mappings

## Implement the random sampling loop for "bad" as done in evoclearn-raw
Steps outlined below:

#### Load the necessary data for the speaker and target sample:

In [4]:
bounds = load_bounds(open(path.join(etcdir, "JD2_bounds.json")))
target_cv_durations = read_durations_from_textgrid(path.join(datadir, "bad.TextGrid"), tier="1")[:2]
target_regions_of_interest = read_intervals_from_textgrid(path.join(datadir, "bad.TextGrid"), tier="1")[:2]
target_sound, target_samplerate = librosa.load(path.join(datadir, "bad.wav"))
# The default audio samplerate is now 44100Hz (aligned with VTL API), so we need to explicitly specify if our target sound has a different samplerate:
target_features = mappings.get_features(target_sound, audio_samplerate=target_samplerate, roi=target_regions_of_interest)

#### Set up some of the functions we will use

In [5]:
add_slope_and_duration = functools.partial(mappings.add_slope_and_duration, durations=target_cv_durations)
get_error = functools.partial(mappings.get_error, ref_feat=target_features)

#### Set up the simulation pipeline:

In [6]:
sampler = UniformProbLink(bounds, consonants=["b"], seed=3)
random_samples = KeepLastValue(sampler)
full_targets = map(add_slope_and_duration, random_samples)
vtl_curves = map(mappings.target_approximation, full_targets)
speech_sounds = KeepLastValue(map(mappings.synthesise_vtl_curves, vtl_curves))
features = map(mappings.get_features, speech_sounds)
errors = map(get_error, features)

#### Define the simulation loop with stopping criteria, and save updates to files as needed:

In [7]:
twenty_iterations = itertools.islice(errors, 20)

best_err = float("inf")
for i, err in enumerate(twenty_iterations):
  print(err)
  if err < best_err:
    print("Found better sample... (saving wave, targets and error)")
    best_err = err
    wav_write(speech_sounds.last_value, f"iteration_{i}.wav")
    with open(f"iteration_{i}.targets.json", "w") as outf:
      outf.write(random_samples.last_value.to_json())
    with open(f"iteration_{i}.error.txt", "w") as outf:
      outf.write(str(err))


48539.21484375
Found better sample... (saving wave, targets and error)
48571.72265625
48858.57421875
32777.92578125
Found better sample... (saving wave, targets and error)
48980.7578125
15641.6435546875
Found better sample... (saving wave, targets and error)
33451.0546875
17516.068359375
49452.1484375
25423.06640625
49409.34375
48683.984375
49060.05859375
25841.9609375
50096.5
48449.98828125
50673.5
49931.3828125
49231.65625
48480.30078125
